# Phase 3: Dual-Branch Meta-Labeling Model

## Overview
This notebook implements a dual-branch neural network architecture for meta-labeling:
- **Branch 1 (Classification)**: Predicts entry signal confidence (should we take this trade?)
- **Branch 2 (Regression)**: Predicts outcome heatmap (signed_heatmap_entry: [-1, +1])

## Architecture
Inspired by swim_code dual-branch CNN-LSTM:
- Shared CNN layers for feature extraction
- Branch 1: CNN → Flatten → Dense → Sigmoid (binary classification)
- Branch 2: CNN → LSTM → Dense → Tanh (regression to [-1, +1])

## Input/Output
- Input: Sequences (60, n_features) from Phase 2
- Output 1: Entry confidence [0, 1]
- Output 2: Outcome prediction [-1, +1]

---

## 1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import pickle 

# TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.constraints import max_norm

# Sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## 2. Configuration

In [ ]:
# Paths
DATA_DIR = Path('../DATA')
MODEL_DIR = Path('../MODELS')
MODEL_DIR.mkdir(exist_ok=True)

# Random seed for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Model hyperparameters
CONFIG = {
    # Architecture
    'shared_conv_filters': [32, 64, 128],
    'shared_kernel_sizes': [3, 3, 3],
    'shared_strides': [1, 1, 1],
    'shared_pool_sizes': [2, 2, 2],
    
    # Branch 1 (Classification)
    'class_dense_units': [64, 32],
    'class_dropout': 0.3,
    'class_l2': 0.001,
    
    # Branch 2 (Regression)
    'reg_lstm_units': 64,
    'reg_lstm_dropout': 0.2,
    'reg_lstm_recurrent_dropout': 0.2,
    'reg_dense_units': [32],
    'reg_dropout': 0.2,
    'reg_l2': 0.001,
    
    # Training
    'batch_size': 64,
    'epochs': 100,
    'learning_rate': 0.001,
    'patience': 15,
    
    # Loss weights (balance between classification and regression)
    'classification_weight': 0.4,
    'regression_weight': 0.6,
}

print("Configuration loaded:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 3. Load Processed Data from Phase 2

In [ ]:
# Load metadata
with open(DATA_DIR / 'data_metadata.json', 'r') as f:
    metadata = json.load(f)

print("Dataset metadata:")
for key, value in metadata.items():
    print(f"  {key}: {value}")

# Load sequences and targets
print("\nLoading data files...")
X_train = np.load(DATA_DIR / 'X_train_scaled.npy')
y_train = np.load(DATA_DIR / 'y_train.npy')
X_val = np.load(DATA_DIR / 'X_val_scaled.npy')
y_val = np.load(DATA_DIR / 'y_val.npy')
X_test = np.load(DATA_DIR / 'X_test_scaled.npy')
y_test = np.load(DATA_DIR / 'y_test.npy')

print(f"\nData shapes:")
print(f"  X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"  X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"  X_test: {X_test.shape}, y_test: {y_test.shape}")

# Input shape for model
input_shape = X_train.shape[1:]  # (60, n_features)
print(f"\nInput shape: {input_shape}")

## 4. Prepare Dual Targets

For meta-labeling, we need two targets:
1. **Classification**: Binary label (good bet vs bad bet)
   - Good bet: |signed_heatmap_entry| > threshold (strong signal)
   - Bad bet: |signed_heatmap_entry| ≤ threshold (weak signal)
2. **Regression**: Continuous signed_heatmap_entry [-1, +1]

In [ ]:
# Classification threshold: consider trades with |heatmap| > 0.3 as "good bets"
CONFIDENCE_THRESHOLD = 0.3

# Create classification labels (0 = bad bet, 1 = good bet)
y_train_class = (np.abs(y_train) > CONFIDENCE_THRESHOLD).astype(np.float32)
y_val_class = (np.abs(y_val) > CONFIDENCE_THRESHOLD).astype(np.float32)
y_test_class = (np.abs(y_test) > CONFIDENCE_THRESHOLD).astype(np.float32)

# Regression targets (keep as-is)
y_train_reg = y_train.astype(np.float32)
y_val_reg = y_val.astype(np.float32)
y_test_reg = y_test.astype(np.float32)

print(f"Classification target distribution:")
print(f"  Train - Good bets: {y_train_class.mean():.2%}, Bad bets: {(1-y_train_class.mean()):.2%}")
print(f"  Val   - Good bets: {y_val_class.mean():.2%}, Bad bets: {(1-y_val_class.mean()):.2%}")
print(f"  Test  - Good bets: {y_test_class.mean():.2%}, Bad bets: {(1-y_test_class.mean()):.2%}")

print(f"\nRegression target statistics:")
print(f"  Train - Mean: {y_train_reg.mean():.4f}, Std: {y_train_reg.std():.4f}")
print(f"  Val   - Mean: {y_val_reg.mean():.4f}, Std: {y_val_reg.std():.4f}")
print(f"  Test  - Mean: {y_test_reg.mean():.4f}, Std: {y_test_reg.std():.4f}")

## 5. Build Dual-Branch Model Architecture

In [ ]:
def build_dual_branch_model(input_shape, config):
    """
    Build dual-branch model for meta-labeling:
    - Branch 1: Classification (entry confidence)
    - Branch 2: Regression (outcome prediction)
    
    Architecture inspired by swim_code dual-branch CNN-LSTM.
    """
    # Input layer
    inputs = layers.Input(shape=input_shape, name='input')
    
    # Reshape for Conv2D: (batch, timesteps, features) → (batch, timesteps, features, 1)
    x = layers.Reshape((input_shape[0], input_shape[1], 1), name='reshape_input')(inputs)
    
    # ===== Shared CNN Layers for Feature Extraction =====
    for i, (filters, kernel_size, stride, pool_size) in enumerate(zip(
        config['shared_conv_filters'],
        config['shared_kernel_sizes'],
        config['shared_strides'],
        config['shared_pool_sizes']
    )):
        x = layers.Conv2D(
            filters=filters,
            kernel_size=(kernel_size, 1),
            strides=(stride, 1),
            padding='same',
            kernel_initializer='he_normal',
            name=f'shared_conv_{i}'
        )(x)
        x = layers.BatchNormalization(name=f'shared_bn_{i}')(x)
        x = layers.Activation('relu', name=f'shared_activation_{i}')(x)
        if pool_size > 1:
            x = layers.MaxPooling2D(pool_size=(pool_size, 1), name=f'shared_pool_{i}')(x)
    
    shared_features = x
    
    # ===== Branch 1: Classification (Entry Confidence) =====
    # Flatten shared features for classification
    class_branch = layers.Flatten(name='class_flatten')(shared_features)
    
    for i, units in enumerate(config['class_dense_units']):
        class_branch = layers.Dense(
            units=units,
            kernel_regularizer=l2(config['class_l2']),
            kernel_initializer='he_normal',
            name=f'class_dense_{i}'
        )(class_branch)
        class_branch = layers.BatchNormalization(name=f'class_bn_dense_{i}')(class_branch)
        class_branch = layers.Activation('relu', name=f'class_activation_{i}')(class_branch)
        class_branch = layers.Dropout(config['class_dropout'], name=f'class_dropout_{i}')(class_branch)
    
    # Classification output: sigmoid for binary classification
    classification_output = layers.Dense(
        units=1,
        activation='sigmoid',
        name='classification_output'
    )(class_branch)
    
    # ===== Branch 2: Regression (Outcome Prediction) =====
    # Reshape shared features for LSTM: (batch, time, features)
    reg_branch = layers.Reshape(
        (shared_features.shape[1], -1),
        name='reg_reshape_for_lstm'
    )(shared_features)
    
    # Bidirectional LSTM for temporal modeling
    reg_branch = layers.Bidirectional(
        layers.LSTM(
            units=config['reg_lstm_units'],
            return_sequences=False,
            dropout=config['reg_lstm_dropout'],
            recurrent_dropout=config['reg_lstm_recurrent_dropout'],
            kernel_initializer='glorot_uniform',
            name='reg_lstm'
        ),
        name='reg_bidirectional_lstm'
    )(reg_branch)
    
    # Dense layers for regression
    for i, units in enumerate(config['reg_dense_units']):
        reg_branch = layers.Dense(
            units=units,
            kernel_regularizer=l2(config['reg_l2']),
            kernel_initializer='he_normal',
            name=f'reg_dense_{i}'
        )(reg_branch)
        reg_branch = layers.BatchNormalization(name=f'reg_bn_dense_{i}')(reg_branch)
        reg_branch = layers.Activation('relu', name=f'reg_activation_{i}')(reg_branch)
        reg_branch = layers.Dropout(config['reg_dropout'], name=f'reg_dropout_{i}')(reg_branch)
    
    # Regression output: tanh to constrain to [-1, +1]
    regression_output = layers.Dense(
        units=1,
        activation='tanh',
        name='regression_output'
    )(reg_branch)
    
    # ===== Build Model with Dual Outputs =====
    model = Model(
        inputs=inputs,
        outputs=[classification_output, regression_output],
        name='DualBranch_MetaLabeling_Model'
    )
    
    return model

# Build the model
model = build_dual_branch_model(input_shape, CONFIG)
model.summary()

## 6. Compile Model

In [ ]:
# Compile with dual losses and metrics
model.compile(
    optimizer=Adam(learning_rate=CONFIG['learning_rate']),
    loss={
        'classification_output': 'binary_crossentropy',
        'regression_output': 'mse'
    },
    loss_weights={
        'classification_output': CONFIG['classification_weight'],
        'regression_output': CONFIG['regression_weight']
    },
    metrics={
        'classification_output': ['accuracy', tf.keras.metrics.AUC(name='auc')],
        'regression_output': ['mae', 'mse']
    }
)

print("Model compiled successfully!")
print(f"\nLoss configuration:")
print(f"  Classification: binary_crossentropy (weight={CONFIG['classification_weight']})")
print(f"  Regression: MSE (weight={CONFIG['regression_weight']})")

## 7. Setup Training Callbacks

In [ ]:
# Create callbacks
checkpoint_path = MODEL_DIR / 'best_model.keras'
log_dir = MODEL_DIR / 'logs'
log_dir.mkdir(exist_ok=True)

callbacks_list = [
    callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
    ),
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=CONFIG['patience'],
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    ),
    callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1
    ),
    callbacks.CSVLogger(
        MODEL_DIR / 'training_history.csv'
    )
]

print("Callbacks configured:")
for cb in callbacks_list:
    print(f"  - {cb.__class__.__name__}")

## 8. Train Model

In [ ]:
print("Starting training...\n")

history = model.fit(
    X_train,
    {
        'classification_output': y_train_class,
        'regression_output': y_train_reg
    },
    validation_data=(
        X_val,
        {
            'classification_output': y_val_class,
            'regression_output': y_val_reg
        }
    ),
    batch_size=CONFIG['batch_size'],
    epochs=CONFIG['epochs'],
    callbacks=callbacks_list,
    verbose=1
)

print("\nTraining completed!")

## 9. Plot Training History

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Total loss
axes[0, 0].plot(history.history['loss'], label='Train')
axes[0, 0].plot(history.history['val_loss'], label='Validation')
axes[0, 0].set_title('Total Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')


In [ ]:
axes[0, 0].legend()
axes[0, 0].grid(True)

# Classification loss
axes[0, 1].plot(history.history['classification_output_loss'], label='Train')
axes[0, 1].plot(history.history['val_classification_output_loss'], label='Validation')
axes[0, 1].set_title('Classification Loss (Binary Crossentropy)')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Regression loss
axes[0, 2].plot(history.history['regression_output_loss'], label='Train')
axes[0, 2].plot(history.history['val_regression_output_loss'], label='Validation')
axes[0, 2].set_title('Regression Loss (MSE)')
axes[0, 2].set_xlabel('Epoch')
axes[0, 2].set_ylabel('Loss')
axes[0, 2].legend()
axes[0, 2].grid(True)

# Classification accuracy
axes[1, 0].plot(history.history['classification_output_accuracy'], label='Train')
axes[1, 0].plot(history.history['val_classification_output_accuracy'], label='Validation')
axes[1, 0].set_title('Classification Accuracy')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Classification AUC
axes[1, 1].plot(history.history['classification_output_auc'], label='Train')
axes[1, 1].plot(history.history['val_classification_output_auc'], label='Validation')
axes[1, 1].set_title('Classification AUC')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('AUC')
axes[1, 1].legend()
axes[1, 1].grid(True)

# Regression MAE
axes[1, 2].plot(history.history['regression_output_mae'], label='Train')
axes[1, 2].plot(history.history['val_regression_output_mae'], label='Validation')
axes[1, 2].set_title('Regression MAE')
axes[1, 2].set_xlabel('Epoch')
axes[1, 2].set_ylabel('MAE')
axes[1, 2].legend()
axes[1, 2].grid(True)

plt.tight_layout()
plt.savefig(MODEL_DIR / 'training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Training history plot saved to {MODEL_DIR / 'training_history.png'}")

## 10. Evaluate on Test Set

In [ ]:
print("Evaluating on test set...\n")

# Get predictions
y_pred_class, y_pred_reg = model.predict(X_test, verbose=0)

# Flatten predictions
y_pred_class = y_pred_class.flatten()
y_pred_reg = y_pred_reg.flatten()

# ===== Classification Metrics =====
print("="*60)
print("CLASSIFICATION METRICS (Entry Confidence)")
print("="*60)

# Convert probabilities to binary predictions
y_pred_class_binary = (y_pred_class > 0.5).astype(int)

accuracy = accuracy_score(y_test_class, y_pred_class_binary)
precision = precision_score(y_test_class, y_pred_class_binary)
recall = recall_score(y_test_class, y_pred_class_binary)
f1 = f1_score(y_test_class, y_pred_class_binary)

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test_class, y_pred_class_binary)
print(cm)
print(f"\nClassification Report:")
print(classification_report(y_test_class, y_pred_class_binary, target_names=['Bad Bet', 'Good Bet']))

# ===== Regression Metrics =====
print("\n" + "="*60)
print("REGRESSION METRICS (Outcome Prediction)")
print("="*60)

mae = mean_absolute_error(y_test_reg, y_pred_reg)
mse = mean_squared_error(y_test_reg, y_pred_reg)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"MAE:  {mae:.4f}")
print(f"MSE:  {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")

# Direction accuracy (sign agreement)
sign_agreement = np.mean(np.sign(y_test_reg) == np.sign(y_pred_reg))
print(f"\nDirection Accuracy (Sign Agreement): {sign_agreement:.4f}")

## 11. Visualize Predictions

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Classification - Confusion Matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0])
axes[0, 0].set_title('Confusion Matrix (Classification)')
axes[0, 0].set_xlabel('Predicted')
axes[0, 0].set_ylabel('Actual')
axes[0, 0].set_xticklabels(['Bad Bet', 'Good Bet'])
axes[0, 0].set_yticklabels(['Bad Bet', 'Good Bet'])

# 2. Classification - Probability Distribution
axes[0, 1].hist(y_pred_class[y_test_class == 0], bins=50, alpha=0.6, label='Bad Bet (Actual)', color='red')
axes[0, 1].hist(y_pred_class[y_test_class == 1], bins=50, alpha=0.6, label='Good Bet (Actual)', color='green')
axes[0, 1].axvline(0.5, color='black', linestyle='--', label='Decision Threshold')
axes[0, 1].set_title('Classification Probability Distribution')
axes[0, 1].set_xlabel('Predicted Probability')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Regression - Scatter Plot
axes[1, 0].scatter(y_test_reg, y_pred_reg, alpha=0.3, s=10)
axes[1, 0].plot([-1, 1], [-1, 1], 'r--', label='Perfect Prediction')
axes[1, 0].set_title('Regression: Actual vs Predicted')
axes[1, 0].set_xlabel('Actual Heatmap Value')
axes[1, 0].set_ylabel('Predicted Heatmap Value')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_xlim(-1.1, 1.1)
axes[1, 0].set_ylim(-1.1, 1.1)

# 4. Regression - Residual Distribution
residuals = y_test_reg - y_pred_reg
axes[1, 1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1, 1].axvline(0, color='red', linestyle='--', label='Zero Residual')
axes[1, 1].set_title('Regression Residual Distribution')
axes[1, 1].set_xlabel('Residual (Actual - Predicted)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(MODEL_DIR / 'predictions_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Predictions analysis plot saved to {MODEL_DIR / 'predictions_analysis.png'}")

## 12. Meta-Labeling Strategy Analysis

Analyze how the dual outputs work together for meta-labeling:
- Only take trades where classification confidence is high
- Use regression output to size/manage the position

In [ ]:
# Create meta-labeling strategy: only take trades with high classification confidence
confidence_thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]

print("Meta-Labeling Strategy Analysis")
print("=" * 80)
print(f"{'Conf Threshold':<15} {'Trades Taken':<15} {'Avg Heatmap':<15} {'Win Rate':<15} {'Directional Acc':<15}")
print("=" * 80)

for threshold in confidence_thresholds:
    # Select trades where classification confidence > threshold
    selected_mask = y_pred_class > threshold
    
    if selected_mask.sum() == 0:
        print(f"{threshold:<15.2f} {'No trades':<15} {'-':<15} {'-':<15} {'-':<15}")
        continue
    
    # Number of trades taken
    n_trades = selected_mask.sum()
    pct_trades = n_trades / len(y_test_reg) * 100
    
    # Average heatmap value (strength of signal)
    avg_heatmap = np.abs(y_test_reg[selected_mask]).mean()
    
    # Win rate (trades where heatmap is correct direction)
    win_rate = np.mean(np.abs(y_test_reg[selected_mask]) > CONFIDENCE_THRESHOLD)
    
    # Directional accuracy
    dir_acc = np.mean(np.sign(y_test_reg[selected_mask]) == np.sign(y_pred_reg[selected_mask]))
    
    print(f"{threshold:<15.2f} {f'{n_trades} ({pct_trades:.1f}%)':<15} {avg_heatmap:<15.4f} {win_rate:<15.2%} {dir_acc:<15.2%}")

print("=" * 80)

## 13. Save Model and Results

In [ ]:
# Save final model
model.save(MODEL_DIR / 'final_model.keras')
print(f"Model saved to {MODEL_DIR / 'final_model.keras'}")

# Save predictions
np.save(MODEL_DIR / 'y_test_class_true.npy', y_test_class)
np.save(MODEL_DIR / 'y_test_class_pred.npy', y_pred_class)
np.save(MODEL_DIR / 'y_test_reg_true.npy', y_test_reg)
np.save(MODEL_DIR / 'y_test_reg_pred.npy', y_pred_reg)
print("Predictions saved.")

# Save configuration and metrics
results = {
    'config': CONFIG,
    'test_metrics': {
        'classification': {
            'accuracy': float(accuracy),
            'precision': float(precision),
            'recall': float(recall),
            'f1_score': float(f1)
        },
        'regression': {
            'mae': float(mae),
            'mse': float(mse),
            'rmse': float(rmse),
            'r2': float(r2),
            'direction_accuracy': float(sign_agreement)
        }
    },
    'training_info': {
        'epochs_trained': len(history.history['loss']),
        'best_val_loss': float(min(history.history['val_loss'])),
        'final_train_loss': float(history.history['loss'][-1]),
        'final_val_loss': float(history.history['val_loss'][-1])
    }
}

with open(MODEL_DIR / 'results.json', 'w') as f:
    json.dump(results, f, indent=2)
print(f"Results saved to {MODEL_DIR / 'results.json'}")

print("\n" + "="*60)
print("Phase 3 Complete!")
print("="*60)
print("Next: Phase 4 - Custom Loss Functions (ConsistentPeakBFCE, Focal Loss)")